# Creating database schema

In order to use this script in VS Code - first initiate a connection to centralised database 'sales_data'



In [ ]:
from sqlalchemy import create_engine
import yaml
import pandas as pd

In [ ]:
with open('Credentials/local_creds.yaml', 'r') as f:
    local_creds = yaml.safe_load(f)
local_db_engine = create_engine(f"{local_creds['DATABASE_TYPE']}+{local_creds['DBAPI']}://{local_creds['USER']}:{local_creds['PASSWORD']}@{local_creds['HOST']}:{local_creds['PORT']}/{local_creds['DATABASE']}")
local_db_engine.execution_options(isolation_level='AUTOCOMMIT').connect()

## Orders table

Changing column datatypes

Change the data types to correspond to those seen in the table below.

The ? in VARCHAR should be replaced with an integer representing the maximum length of the values in that column.

In [ ]:
+------------------+--------------------+--------------------+
|   orders_table   | current data type  | required data type |
+------------------+--------------------+--------------------+
| date_uuid        | TEXT               | UUID               |
| user_uuid        | TEXT               | UUID               |
| card_number      | TEXT               | VARCHAR(?)         |
| store_code       | TEXT               | VARCHAR(?)         |
| product_code     | TEXT               | VARCHAR(?)         |
| product_quantity | BIGINT             | SMALLINT           |
+------------------+--------------------+--------------------+

In [ ]:
ALTER TABLE orders_table
ALTER COLUMN date_uuid TYPE uuid
USING date_uuid::uuid;

In [ ]:
ALTER TABLE orders_table
ALTER COLUMN user_uuid TYPE uuid
USING user_uuid::uuid;

Using pandas, finding out the maximum value length in each column. 

In [ ]:
#reading in sql table from local database as pandas data frame
clean_orders_df = pd.read_sql_table('orders_table', local_db_engine)

In [ ]:
for column in clean_orders_df:
    print(column,"->", clean_orders_df[column].astype(str).str.len().max())

In [ ]:
ALTER TABLE orders_table
    ALTER COLUMN product_quantity TYPE SMALLINT,
    ALTER COLUMN card_number TYPE VARCHAR(19),
    ALTER COLUMN store_code TYPE VARCHAR(12),
    ALTER COLUMN product_code TYPE VARCHAR(11);

## Users table

In [ ]:
+----------------+--------------------+--------------------+
| dim_user_table | current data type  | required data type |
+----------------+--------------------+--------------------+
| first_name     | TEXT               | VARCHAR(255)       |
| last_name      | TEXT               | VARCHAR(255)       |
| date_of_birth  | TEXT               | DATE               |
| country_code   | TEXT               | VARCHAR(?)         |
| user_uuid      | TEXT               | UUID               |
| join_date      | TEXT               | DATE               |
+----------------+--------------------+--------------------+

In [ ]:
ALTER TABLE dim_users
    ALTER COLUMN first_name TYPE VARCHAR(255), 
    ALTER COLUMN last_name TYPE VARCHAR(255), 
    ALTER COLUMN date_of_birth TYPE DATE, 
    ALTER COLUMN country_code TYPE VARCHAR(3), 
    ALTER COLUMN user_uuid TYPE uuid USING user_uuid::uuid, 
    ALTER COLUMN join_date TYPE DATE;

## Store details

There are two latitude columns in the store details table. Using SQL, merge one of the columns into the other so you have one latitude column.

Then set the data types for each column as shown below:


There is a row that represents the business's website change the location column values where they're null to N/A.

In [ ]:
+---------------------+-------------------+------------------------+
| store_details_table | current data type |   required data type   |
+---------------------+-------------------+------------------------+
| longitude           | TEXT              | FLOAT                  |
| locality            | TEXT              | VARCHAR(255)           |
| store_code          | TEXT              | VARCHAR(?)             |
| staff_numbers       | TEXT              | SMALLINT               |
| opening_date        | TEXT              | DATE                   |
| store_type          | TEXT              | VARCHAR(255) NULLABLE  |
| latitude            | TEXT              | FLOAT                  |
| country_code        | TEXT              | VARCHAR(?)             |
| continent           | TEXT              | VARCHAR(255)           |
+---------------------+-------------------+------------------------+

In [ ]:
UPDATE dim_store_details
    SET locality = 'N/A'
WHERE
    store_type = 'Web Portal';

In [ ]:
ALTER TABLE dim_store_details
ALTER COLUMN longitude TYPE FLOAT USING longitude::FLOAT,
ALTER COLUMN locality TYPE VARCHAR(255),
ALTER COLUMN store_code TYPE VARCHAR(12),
ALTER COLUMN staff_numbers TYPE SMALLINT,
ALTER COLUMN opening_date TYPE DATE,
ALTER COLUMN store_type TYPE VARCHAR(255),
ALTER COLUMN latitude TYPE FLOAT USING latitude::FLOAT,
ALTER COLUMN country_code TYPE VARCHAR(3),
ALTER COLUMN continent TYPE VARCHAR(255);

## Product details

You will need to do some work on the products table before casting the data types correctly.

The product_price column has a £ character which you need to remove using SQL.

The team that handles the deliveries would like a new human-readable column added for the weight so they can quickly make decisions on delivery weights.

Add a new column weight_class which will contain human-readable values based on the weight range of the product.

In [ ]:
+--------------------------+-------------------+
| weight_class VARCHAR(?)  | weight range(kg)  |
+--------------------------+-------------------+
| Light                    | < 2               |
| Mid_Sized                | >= 2 - < 40       |
| Heavy                    | >= 40 - < 140     |
| Truck_Required           | => 140            |
+----------------------------+-----------------+

In [ ]:
UPDATE
  dim_products
SET product_price = REGEXP_REPLACE(product_price, '£', '')

Need to add the column, THEN update column based on conditionals

In [ ]:
ALTER TABLE dim_products
    ADD COLUMN weight_class VARCHAR(14);

In [ ]:
UPDATE dim_products
SET
  weight_class = CASE
    WHEN weight < 2 THEN 'Light'
    WHEN weight >= 2
    AND weight < 40 THEN 'Mid_Sized'
    WHEN weight >= 40
    AND weight < 140 THEN 'Heavy'
    ELSE 'Truck_required'
  END;

After all the columns are created and cleaned, change the data types of the products table.

You will want to rename the removed column to still_available before changing its data type.

Make the changes to the columns to cast them to the following data types:

In [ ]:
+-----------------+--------------------+--------------------+
|  dim_products   | current data type  | required data type |
+-----------------+--------------------+--------------------+
| product_price   | TEXT               | FLOAT              |
| weight          | TEXT               | FLOAT              |
| EAN             | TEXT               | VARCHAR(?)         |
| product_code    | TEXT               | VARCHAR(?)         |
| date_added      | TEXT               | DATE               |
| uuid            | TEXT               | UUID               |
| still_available | TEXT               | BOOL               |
| weight_class    | TEXT               | VARCHAR(?)         |
+-----------------+--------------------+--------------------+

In [ ]:
ALTER TABLE dim_products
    RENAME removed TO still_available;

In [ ]:
prod_df = pd.read_sql_table('dim_products', local_db_engine)

for column in prod_df:
    print(column,"->", prod_df[column].astype(str).str.len().max())

In [ ]:
ALTER TABLE dim_products
    ALTER COLUMN product_price TYPE FLOAT USING product_price::FLOAT,
    ALTER COLUMN weight TYPE FLOAT,
    ALTER COLUMN "EAN" TYPE VARCHAR(17),
    ALTER COLUMN product_code TYPE VARCHAR(11),
    ALTER COLUMN date_added TYPE DATE,
    ALTER COLUMN uuid TYPE UUID USING uuid::UUID,
    ALTER COLUMN still_available TYPE BOOLEAN USING CASE WHEN still_available = 'Still_avaliable' THEN TRUE ELSE FALSE END,
    ALTER weight_class TYPE VARCHAR(14);

## Date times

In [ ]:
+-----------------+-------------------+--------------------+
| dim_date_times  | current data type | required data type |
+-----------------+-------------------+--------------------+
| month           | TEXT              | VARCHAR(?)         |
| year            | TEXT              | VARCHAR(?)         |
| day             | TEXT              | VARCHAR(?)         |
| time_period     | TEXT              | VARCHAR(?)         |
| date_uuid       | TEXT              | UUID               |
+-----------------+-------------------+--------------------+

In [ ]:
date_times_df = pd.read_sql_table('dim_date_times', local_db_engine)

for column in date_times_df:
    print(column,"->", date_times_df[column].astype(str).str.len().max())

In [ ]:
date_times_df.head()

In [ ]:
ALTER TABLE dim_date_times
    ALTER COLUMN month TYPE VARCHAR(2),
    ALTER COLUMN year TYPE VARCHAR(4),
    ALTER COLUMN day TYPE VARCHAR(2),
    ALTER COLUMN time_period TYPE VARCHAR(10),
    ALTER COLUMN date_uuid TYPE UUID USING date_uuid::UUID;


Now we need to update the last table for the card details.

Make the associated changes after finding out what the lengths of each variable should be:

In [ ]:
+------------------------+-------------------+--------------------+
|    dim_card_details    | current data type | required data type |
+------------------------+-------------------+--------------------+
| card_number            | TEXT              | VARCHAR(?)         |
| expiry_date            | TEXT              | VARCHAR(?)         |
| date_payment_confirmed | TEXT              | DATE               |
+------------------------+-------------------+--------------------+

In [ ]:
card_df = pd.read_sql_table('dim_card_details', local_db_engine)

for column in card_df:
    print(column,"->", card_df[column].astype(str).str.len().max())

In [ ]:
ALTER TABLE dim_card_details
    ALTER COLUMN card_number TYPE VARCHAR(19),
    ALTER COLUMN expiry_date TYPE VARCHAR(5),
    ALTER COLUMN date_payment_confirmed TYPE DATE;

# Keys

Now that the tables have the appropriate data types we can begin adding the primary keys to each of the tables prefixed with dim.

Each table will serve the orders_table which will be the single source of truth for our orders.

Check the column header of the orders_table you will see all but one of the columns exist in one of our tables prefixed with dim.

We need to update the columns in the dim tables with a primary key that matches the same column in the orders_table.

Using SQL, update the respective columns as primary key columns.

In [ ]:
ALTER TABLE dim_card_details
    ADD PRIMARY KEY (card_number);

In [ ]:
ALTER TABLE dim_date_times
    ADD PRIMARY KEY (date_uuid);

In [ ]:
ALTER TABLE dim_products
    ADD PRIMARY KEY (product_code);

In [ ]:
ALTER TABLE dim_store_details
    ADD PRIMARY KEY (store_code);

In [ ]:
ALTER TABLE dim_users
    ADD PRIMARY KEY (user_uuid);

## Foreign keys

Keep getting a violation of constraints for creating foreign keys. 

This can be due to rows in the orders_table not matching the rows in your other table.

A solution would be to do a left join and see where any nulls are created. 

In [ ]:
SELECT orders_table.card_number,
    orders_table.index,
    dim_card_details.index,
    dim_card_details.card_number,
    dim_card_details.card_provider,
    dim_card_details.expiry_date,
    dim_card_details.date_payment_confirmed
FROM
    orders_table
LEFT OUTER JOIN
    dim_card_details ON orders_table.card_number = dim_card_details.card_number


Upon cross-checking NULL values, I managed to find the issue was during the cleaning methods of the card details dataframe after extraction. Some cleaning methods were not behaving as expected. 

In [ ]:
SELECT orders_table.product_code,
    orders_table.index,
    dim_products.index,
    dim_products.product_code,
    dim_products.product_name,
    dim_products.category
FROM
    orders_table
LEFT OUTER JOIN
    dim_products ON orders_table.product_code = dim_products.product_code

In [ ]:
SELECT orders_table.store_code,
    orders_table.index,
    dim_store_details.index,
    dim_store_details.store_code,
    dim_store_details.store_type,
    dim_store_details.country_code
FROM
    orders_table
LEFT OUTER JOIN
    dim_store_details ON orders_table.store_code = dim_store_details.store_code

In [ ]:
ALTER TABLE orders_table
ADD FOREIGN KEY (card_number) REFERENCES dim_card_details(card_number),
ADD FOREIGN KEY (date_uuid) REFERENCES dim_date_times(date_uuid),
ADD FOREIGN KEY (product_code) REFERENCES dim_products(product_code),
ADD FOREIGN KEY (store_code) REFERENCES dim_store_details(store_code),
ADD FOREIGN KEY (user_uuid) REFERENCES dim_users(user_uuid)
ON DELETE CASCADE
ON UPDATE CASCADE;